In [ ]:
mem = [0]*100

A=0
B=0
PC=0
CIR=0
AR=0
DR=0
INTR=0

#INTERRUPT CODES
INTR_OK  =0
INTR_HALT=1
INTR_ILLI=2

#INSTRUCTION SET
isa=[ "LOADA", "LOADB", "STOREA", "STOREB", "ADD", "DIF", "JMP", "JMPZ", "NOP", "HALT", "READ", "WRITE", ]

def encode_instr(fcode, operand):
	return (isa.index(fcode)<<12)+operand

def decode_instr(instr):
	return isa[instr>>12], instr%(2**12)

def encode_program(p):
	global mem
	i=0
	for fc, op in p : 
		mem[i]=encode_instr(fc, op)
		i+=1


def io_read():
	r=raw_input("INPUT >> ")
	return int(r)

def io_write(value):
	print("OUTPUT >> ", value)

def fetch_decode_execute():
	global PC
	global INTR
	global A
	global B
	global AR
	global DR
	global CIR
	#FETCH
	AR=PC
	DR=mem[AR]
	CIR=DR
	PC=PC+1
	#DECODE
	fcode, op = decode_instr(CIR)
	print("Executing:", fcode, op, "from address", AR)
	#EXECUTE
	if fcode=="LOADA" : 
		AR=op
		DR=mem[AR]
		A=DR
	elif fcode=="LOADB" : 
		AR=op
		DR=mem[AR]
		B=DR
	elif fcode=="STOREA" : 
		AR=op
		DR=A
		mem[AR]=DR
	elif fcode=="STOREB" : 
		AR=op
		DR=B
		mem[AR]=DR
	elif fcode=="ADD" : 
		A=A+B
	elif fcode=="DIF" : 
		A=A-B
	elif fcode=="JMP" : 
		PC=op
	elif fcode=="JMPZ" : 
		if not A : PC=op
	elif fcode=="NOP" :
		pass
	elif fcode=="HALT" :
		INTR=INTR_HALT
	elif fcode=="READ" :
		DR=io_read()
		AR=op
		mem[AR]=DR
	elif fcode=="WRITE" :
		AR=op
		DR=mem[AR]
		io_write(DR)
	else :
		INTR=INTR_ILLI
	

def loop():
	while(INTR==0):
		fetch_decode_execute()
		print("PC={}, INTR={}, A={}, B={}".format(PC, INTR, A, B))
	if INTR==INTR_HALT : print("Shutdown")
	elif INTR==INTR_ILLI : print("Illegal instruction")

def compile_program():
	fc="NOP"
	i=0
	p=[]
	varlist = []
	while(fc!="HALT"):
		line=raw_input("{}: ".format(i))
		try : fc, op = tuple(line.split()[:2])
		except ValueError : 
			fc = line
			op = 0
		try : op=int(op) 
		except Exception :
			if op not in varlist : varlist.append(op)
		p+=[(fc,op)]
		i=i+1
	def replace(varname, current, addr):
		if varname==current : return addr
		return varname
	for x in varlist :
		j=varlist.index(x)+i
		p=[ (fc, replace(op,x,j)) for fc, op in p ]
	encode_program(p)
	print("memory")
	for j in range(i) :
		print(hex(mem[j]))

def run():
	compile_program()
	loop()

run()